In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from numpy.random import default_rng
from collections import defaultdict
import simpy as sp

plt.rcParams['text.usetex'] = True

In [ ]:
def harmonic(N, s):
    harmonic_sum = 0
    for k in range(1,N+1):
        harmonic_sum += 1/(k**s)
    return harmonic_sum

def zipf(k, N, s):
    return 1 / (harmonic(N,s) * k**s)

def avgPackets(cap,rate):
    return rate/(cap-rate)

def offlineReqGen(N, R, zipf_param, exp_rate):
    rng = default_rng(123)
    prob_dist = [zipf(k, N, zipf_param) for k in range(1,N+1)]
    intervals = []
    objects = []
    timestamps = defaultdict(list)
    r = 0
    while r < R:
        intervals.append(rng.exponential(1/exp_rate))
        objects.append(rng.choice(np.arange(1,N+1), p = prob_dist))
        r += 1
    interval_cumsum = np.cumsum(intervals)
    for ts, obj in zip(interval_cumsum, objects):
        timestamps[obj].append(ts)
    for obj in timestamps:
        timestamps[obj].reverse()
    return intervals, timestamps, objects


In [29]:
env = sp.Environment()
serverLink = sp.Resource(env, 1)
linkRate = 100
serverDelay = 0
cacheLink = sp.Resource(env, 1)
cache = []
cacheRate = 100
cacheCap = 50
cacheDelay = 0
intervals, timestamps, objects = offlineReqGen(1000, 5000, 0.75, 100)

def serverProcess(obj):
    global serverDelay
    with serverLink.request() as request:
        tic = env.now
        yield request
        yield env.timeout(1/linkRate)
        toc = env.now
        cacheDecision(obj)
        serverDelay += toc - tic
    return

def cacheProcess():
    global cacheDelay
    with cacheLink.request() as request:
        tic = env.now
        yield request
        yield env.timeout(1/cacheRate)
        toc = env.now
        cacheDelay += toc - tic
    return

def cacheDecision(obj):
    if len(cache) < cacheCap:
        cache.append(obj)
        return
    victim_timestamp = 0
    victim = None
    for pot_victim in cache:
        if len(timestamps[pot_victim]) == 0:
            victim = pot_victim
            break
        next_timestamp = timestamps[pot_victim][-1]
        if next_timestamp > victim_timestamp:
            victim = pot_victim
            victim_timestamp = next_timestamp
    cache.remove(victim)
    cache.append(obj)

def requestProcess():
    for interval, obj in zip(intervals, objects):
        yield env.timeout(interval)
        ts = timestamps[obj].pop()
        if obj in cache:
            env.process(cacheProcess())
        else:
            env.process(serverProcess(obj))

env.process(requestProcess())
env.run(until=150)

print(cacheDelay, serverDelay)

36.576795369748815 43.36619374120521
